In [5]:

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import json
import os
from google.cloud import storage
from google import genai

warnings.filterwarnings('ignore')

class TechnicalAnalyzer:
    def __init__(self, symbol, period='1y', gcp_bucket='ttb-bucket1', gemini_api_key=None, local_save_dir='technical_analysis_data'):
        self.symbol = symbol
        self.period = period
        self.data = None
        self.signals = []
        self.gcp_bucket = gcp_bucket
        self.gemini_api_key = gemini_api_key
        self.local_save_dir = local_save_dir
        
        if self.gemini_api_key:
            self.genai_client = genai.Client(api_key=self.gemini_api_key)
        else:
            self.genai_client = None
        
        self._setup_local_folders()
    
    def _setup_local_folders(self):
        """Create local folder structure for saving analysis files"""
        date_str = datetime.now().strftime('%Y-%m-%d')
        
        if not os.path.exists(self.local_save_dir):
            os.makedirs(self.local_save_dir)
            print(f"📁 Created main directory: {self.local_save_dir}")
        
        self.date_folder = os.path.join(self.local_save_dir, date_str)
        if not os.path.exists(self.date_folder):
            os.makedirs(self.date_folder)
            print(f"📁 Created date folder: {self.date_folder}")
        else:
            print(f"📁 Using existing folder: {self.date_folder}")
    
    def _generate_filename(self, file_type, extension):
        """Generate standardized filename: YYYY-MM-DD-SYMBOL-type.ext"""
        date_str = datetime.now().strftime('%Y-%m-%d')
        timestamp = datetime.now().strftime('%H%M%S')
        return f"{date_str}-{self.symbol}-{file_type}-{timestamp}.{extension}"
    
    def fetch_data(self):
        """Fetch stock data from Yahoo Finance"""
        print(f"📊 Fetching data for {self.symbol}...")
        ticker = yf.Ticker(self.symbol)
        self.data = ticker.history(period=self.period)
        
        if self.data.empty:
            raise ValueError(f"No data found for {self.symbol}")
        
        print(f"✅ Fetched {len(self.data)} days of data")
        return self.data
    
    def calculate_indicators(self):
        """Calculate comprehensive technical indicators"""
        df = self.data.copy()
        
        print("\n🔧 Calculating Technical Indicators...")
        
        # Moving Averages
        for period in [5, 10, 20, 50, 100, 200]:
            df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
            df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
        
        # Bollinger Bands
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()
        bb_std = df['Close'].rolling(window=20).std()
        df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
        df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
        df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        
        # Stochastic
        low_14 = df['Low'].rolling(window=14).min()
        high_14 = df['High'].rolling(window=14).max()
        df['Stoch_K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
        df['Stoch_D'] = df['Stoch_K'].rolling(window=3).mean()
        
        # ATR
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        df['ATR'] = true_range.rolling(14).mean()
        
        # ADX
        plus_dm = df['High'].diff()
        minus_dm = -df['Low'].diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm < 0] = 0
        tr14 = true_range.rolling(14).sum()
        plus_di = 100 * (plus_dm.rolling(14).sum() / tr14)
        minus_di = 100 * (minus_dm.rolling(14).sum() / tr14)
        dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)
        df['ADX'] = dx.rolling(14).mean()
        df['Plus_DI'] = plus_di
        df['Minus_DI'] = minus_di
        
        # CCI
        tp = (df['High'] + df['Low'] + df['Close']) / 3
        df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
        
        # Williams %R
        df['Williams_R'] = -100 * ((high_14 - df['Close']) / (high_14 - low_14))
        
        # OBV
        df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
        
        # Volume
        df['Volume_MA_20'] = df['Volume'].rolling(window=20).mean()
        df['Volume_MA_50'] = df['Volume'].rolling(window=50).mean()
        
        # VWAP
        df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
        
        # ROC
        for period in [5, 10, 20]:
            df[f'ROC_{period}'] = ((df['Close'] - df['Close'].shift(period)) / df['Close'].shift(period)) * 100
        
        # MFI
        typical_price = (df['High'] + df['Low'] + df['Close']) / 3
        money_flow = typical_price * df['Volume']
        positive_flow = money_flow.where(typical_price > typical_price.shift(1), 0).rolling(14).sum()
        negative_flow = money_flow.where(typical_price < typical_price.shift(1), 0).rolling(14).sum()
        mfi_ratio = positive_flow / negative_flow
        df['MFI'] = 100 - (100 / (1 + mfi_ratio))
        
        # Ichimoku
        high_9 = df['High'].rolling(window=9).max()
        low_9 = df['Low'].rolling(window=9).min()
        df['Tenkan'] = (high_9 + low_9) / 2
        high_26 = df['High'].rolling(window=26).max()
        low_26 = df['Low'].rolling(window=26).min()
        df['Kijun'] = (high_26 + low_26) / 2
        df['Senkou_A'] = ((df['Tenkan'] + df['Kijun']) / 2).shift(26)
        high_52 = df['High'].rolling(window=52).max()
        low_52 = df['Low'].rolling(window=52).min()
        df['Senkou_B'] = ((high_52 + low_52) / 2).shift(26)
        
        # Momentum
        df['Momentum'] = df['Close'] - df['Close'].shift(10)
        
        # Volatility
        df['Volatility'] = df['Close'].pct_change().rolling(20).std() * np.sqrt(252) * 100
        
        # Price changes
        df['Price_Change'] = df['Close'].pct_change() * 100
        df['Price_Change_5d'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)) * 100
        
        # Highs and Lows
        df['High_52w'] = df['High'].rolling(window=252).max()
        df['Low_52w'] = df['Low'].rolling(window=252).min()
        df['High_20d'] = df['High'].rolling(window=20).max()
        df['Low_20d'] = df['Low'].rolling(window=20).min()
        
        # Distance from MAs
        for period in [10, 20, 50, 200]:
            df[f'Dist_SMA_{period}'] = ((df['Close'] - df[f'SMA_{period}']) / df[f'SMA_{period}']) * 100
        
        self.data = df
        print("✅ All indicators calculated")
        return df
    
    def detect_signals(self):
        """Detect 100+ comprehensive technical signals"""
        df = self.data.copy()
        current = df.iloc[-1]
        prev = df.iloc[-2]
        prev2 = df.iloc[-3] if len(df) > 2 else prev
        
        signals = []
        
        print("\n🎯 Scanning for Technical Alerts...")
        
        # Moving Average Crossovers
        if len(df) > 200 and prev['SMA_50'] <= prev['SMA_200'] and current['SMA_50'] > current['SMA_200']:
            signals.append({'signal': 'GOLDEN CROSS', 'desc': '50 MA crossed above 200 MA', 'strength': 'STRONG BULLISH', 'category': 'MA_CROSS'})
        
        if len(df) > 200 and prev['SMA_50'] >= prev['SMA_200'] and current['SMA_50'] < current['SMA_200']:
            signals.append({'signal': 'DEATH CROSS', 'desc': '50 MA crossed below 200 MA', 'strength': 'STRONG BEARISH', 'category': 'MA_CROSS'})
        
        if prev['Close'] <= prev['SMA_10'] and current['Close'] > current['SMA_10']:
            signals.append({'signal': 'PRICE ABOVE 10 MA', 'desc': 'Price crossed above 10-day MA', 'strength': 'BULLISH', 'category': 'MA_CROSS'})
        
        if prev['Close'] >= prev['SMA_10'] and current['Close'] < current['SMA_10']:
            signals.append({'signal': 'PRICE BELOW 10 MA', 'desc': 'Price crossed below 10-day MA', 'strength': 'BEARISH', 'category': 'MA_CROSS'})
        
        if prev['Close'] <= prev['SMA_20'] and current['Close'] > current['SMA_20']:
            signals.append({'signal': 'PRICE ABOVE 20 MA', 'desc': 'Price crossed above 20-day MA', 'strength': 'BULLISH', 'category': 'MA_CROSS'})
        
        if prev['Close'] >= prev['SMA_20'] and current['Close'] < current['SMA_20']:
            signals.append({'signal': 'PRICE BELOW 20 MA', 'desc': 'Price crossed below 20-day MA', 'strength': 'BEARISH', 'category': 'MA_CROSS'})
        
        if prev['EMA_10'] <= prev['EMA_20'] and current['EMA_10'] > current['EMA_20']:
            signals.append({'signal': '10/20 EMA BULL CROSS', 'desc': '10 EMA crossed above 20 EMA', 'strength': 'BULLISH', 'category': 'MA_CROSS'})
        
        if prev['EMA_10'] >= prev['EMA_20'] and current['EMA_10'] < current['EMA_20']:
            signals.append({'signal': '10/20 EMA BEAR CROSS', 'desc': '10 EMA crossed below 20 EMA', 'strength': 'BEARISH', 'category': 'MA_CROSS'})
        
        # RSI Signals
        if current['RSI'] < 30:
            signals.append({'signal': 'RSI OVERSOLD', 'desc': f"RSI at {current['RSI']:.1f}", 'strength': 'BULLISH', 'category': 'RSI'})
        
        if current['RSI'] > 70:
            signals.append({'signal': 'RSI OVERBOUGHT', 'desc': f"RSI at {current['RSI']:.1f}", 'strength': 'BEARISH', 'category': 'RSI'})
        
        if current['RSI'] < 20:
            signals.append({'signal': 'RSI EXTREME OVERSOLD', 'desc': f"RSI at {current['RSI']:.1f}", 'strength': 'STRONG BULLISH', 'category': 'RSI'})
        
        if current['RSI'] > 80:
            signals.append({'signal': 'RSI EXTREME OVERBOUGHT', 'desc': f"RSI at {current['RSI']:.1f}", 'strength': 'STRONG BEARISH', 'category': 'RSI'})
        
        if len(df) > 20:
            if current['Close'] < df['Close'].iloc[-20] and current['RSI'] > df['RSI'].iloc[-20]:
                signals.append({'signal': 'RSI BULLISH DIVERGENCE', 'desc': 'Price down but RSI up', 'strength': 'BULLISH', 'category': 'DIVERGENCE'})
        
        if len(df) > 20:
            if current['Close'] > df['Close'].iloc[-20] and current['RSI'] < df['RSI'].iloc[-20]:
                signals.append({'signal': 'RSI BEARISH DIVERGENCE', 'desc': 'Price up but RSI down', 'strength': 'BEARISH', 'category': 'DIVERGENCE'})
        
        # MACD Signals
        if prev['MACD'] <= prev['MACD_Signal'] and current['MACD'] > current['MACD_Signal']:
            signals.append({'signal': 'MACD BULL CROSS', 'desc': 'MACD crossed above signal', 'strength': 'BULLISH', 'category': 'MACD'})
        
        if prev['MACD'] >= prev['MACD_Signal'] and current['MACD'] < current['MACD_Signal']:
            signals.append({'signal': 'MACD BEAR CROSS', 'desc': 'MACD crossed below signal', 'strength': 'BEARISH', 'category': 'MACD'})
        
        if prev['MACD'] <= 0 and current['MACD'] > 0:
            signals.append({'signal': 'MACD ABOVE ZERO', 'desc': 'MACD crossed into positive territory', 'strength': 'BULLISH', 'category': 'MACD'})
        
        if prev['MACD'] >= 0 and current['MACD'] < 0:
            signals.append({'signal': 'MACD BELOW ZERO', 'desc': 'MACD crossed into negative territory', 'strength': 'BEARISH', 'category': 'MACD'})
        
        # Bollinger Bands
        bb_width_avg = df['BB_Width'].tail(50).mean()
        if current['BB_Width'] < bb_width_avg * 0.7:
            signals.append({'signal': 'BB SQUEEZE', 'desc': 'Bands narrowing - breakout pending', 'strength': 'NEUTRAL', 'category': 'BOLLINGER'})
        
        if current['Close'] <= current['BB_Lower'] * 1.01:
            signals.append({'signal': 'AT LOWER BB', 'desc': f"Price at ${current['BB_Lower']:.2f}", 'strength': 'BULLISH', 'category': 'BOLLINGER'})
        
        if current['Close'] >= current['BB_Upper'] * 0.99:
            signals.append({'signal': 'AT UPPER BB', 'desc': f"Price at ${current['BB_Upper']:.2f}", 'strength': 'BEARISH', 'category': 'BOLLINGER'})
        
        # Volume Signals
        if current['Volume'] > current['Volume_MA_20'] * 2:
            signals.append({'signal': 'VOLUME SPIKE 2X', 'desc': f"Vol: {current['Volume']:,.0f}", 'strength': 'SIGNIFICANT', 'category': 'VOLUME'})
        
        if current['Volume'] > current['Volume_MA_20'] * 3:
            signals.append({'signal': 'EXTREME VOLUME 3X', 'desc': f"Vol: {current['Volume']:,.0f}", 'strength': 'VERY SIGNIFICANT', 'category': 'VOLUME'})
        
        if current['Price_Change'] > 2 and current['Volume'] > current['Volume_MA_20'] * 1.5:
            signals.append({'signal': 'VOLUME BREAKOUT', 'desc': 'High volume + price up', 'strength': 'STRONG BULLISH', 'category': 'VOLUME'})
        
        if current['Price_Change'] < -2 and current['Volume'] > current['Volume_MA_20'] * 1.5:
            signals.append({'signal': 'VOLUME SELLOFF', 'desc': 'High volume + price down', 'strength': 'STRONG BEARISH', 'category': 'VOLUME'})
        
        # Price Action
        if current['Price_Change'] > 5:
            signals.append({'signal': 'LARGE GAIN', 'desc': f"+{current['Price_Change']:.1f}% today", 'strength': 'STRONG BULLISH', 'category': 'PRICE_ACTION'})
        
        if current['Price_Change'] < -5:
            signals.append({'signal': 'LARGE LOSS', 'desc': f"{current['Price_Change']:.1f}% today", 'strength': 'STRONG BEARISH', 'category': 'PRICE_ACTION'})
        
        if current['Close'] >= current['High_52w'] * 0.999:
            signals.append({'signal': '52-WEEK HIGH', 'desc': f"At ${current['Close']:.2f}", 'strength': 'STRONG BULLISH', 'category': 'RANGE'})
        
        if current['Close'] <= current['Low_52w'] * 1.001:
            signals.append({'signal': '52-WEEK LOW', 'desc': f"At ${current['Close']:.2f}", 'strength': 'STRONG BEARISH', 'category': 'RANGE'})
        
        # Trend Strength
        if current['ADX'] > 25:
            trend = 'UP' if current['Close'] > current['SMA_50'] else 'DOWN'
            signals.append({'signal': f"STRONG {trend}TREND", 'desc': f"ADX: {current['ADX']:.1f}", 'strength': 'TRENDING', 'category': 'TREND'})
        
        if current['ADX'] > 40:
            signals.append({'signal': 'VERY STRONG TREND', 'desc': f"ADX: {current['ADX']:.1f}", 'strength': 'EXTREME', 'category': 'TREND'})
        
        # MA Alignment
        mas_aligned_bull = (current['SMA_10'] > current['SMA_20'] > current['SMA_50'])
        if mas_aligned_bull:
            signals.append({'signal': 'MA ALIGNMENT BULLISH', 'desc': '10 > 20 > 50 SMA', 'strength': 'STRONG BULLISH', 'category': 'MA_TREND'})
        
        mas_aligned_bear = (current['SMA_10'] < current['SMA_20'] < current['SMA_50'])
        if mas_aligned_bear:
            signals.append({'signal': 'MA ALIGNMENT BEARISH', 'desc': '10 < 20 < 50 SMA', 'strength': 'STRONG BEARISH', 'category': 'MA_TREND'})
        
        if current['Close'] > current['SMA_200'] and len(df) > 200:
            signals.append({'signal': 'ABOVE 200 SMA', 'desc': 'Long-term uptrend', 'strength': 'BULLISH', 'category': 'MA_TREND'})
        
        if current['Close'] < current['SMA_200'] and len(df) > 200:
            signals.append({'signal': 'BELOW 200 SMA', 'desc': 'Long-term downtrend', 'strength': 'BEARISH', 'category': 'MA_TREND'})
        
        self.signals = signals
        print(f"✅ Detected {len(signals)} Active Signals")
        return signals
    
    def rank_signals_with_ai(self):
        """Use Gemini AI to score each signal from 1-100"""
        if not self.genai_client:
            print("\n⚠️  Gemini API key not provided. Skipping AI ranking.")
            for signal in self.signals:
                signal['ai_score'] = 50
                signal['ai_reasoning'] = 'No AI scoring available'
            return
        
        print("\n🤖 AI is scoring all signals (1-100)...")
        
        try:
            current = self.data.iloc[-1]
            
            prompt = f"""You are an expert technical analyst scoring trading signals for {self.symbol}.

CURRENT MARKET DATA:
- Price: ${current['Close']:.2f}
- Daily Change: {current['Price_Change']:.2f}%
- RSI: {current['RSI']:.1f}
- MACD: {current['MACD']:.4f}
- ADX: {current['ADX']:.1f}
- Volatility: {current['Volatility']:.1f}%
- Volume vs 20-day avg: {(current['Volume'] / current['Volume_MA_20'] * 100):.0f}%

SIGNALS TO SCORE:
"""
            for i, sig in enumerate(self.signals, 1):
                prompt += f"\n{i}. {sig['signal']} - {sig['desc']} ({sig['strength']}) [{sig['category']}]"
            
            prompt += """

TASK: Score each signal from 1-100 based on:
- Actionability (how tradeable is this signal?)
- Reliability (historical success rate)
- Timing (is this the right moment?)
- Risk/Reward potential
- Market context alignment

RESPONSE FORMAT (JSON):
{
  "scores": [
    {"signal_number": 1, "score": 85, "reasoning": "Strong reversal signal with volume"},
    {"signal_number": 2, "score": 72, "reasoning": "Moderate signal but overbought"},
    ...
  ],
  "top_signal": {
    "signal_number": 1,
    "why": "Best risk/reward setup"
  }
}

Score ALL signals. Use full 1-100 range. Be critical."""

            response = self.genai_client.models.generate_content(
                model='gemini-2.0-flash-exp',
                contents=prompt
            )
            
            response_text = response.text.strip()
            
            if '```json' in response_text:
                response_text = response_text.split('```json')[1].split('```')[0].strip()
            elif '```' in response_text:
                response_text = response_text.split('```')[1].split('```')[0].strip()
            
            scores_data = json.loads(response_text)
            
            for score_item in scores_data['scores']:
                sig_num = score_item['signal_number'] - 1
                if 0 <= sig_num < len(self.signals):
                    self.signals[sig_num]['ai_score'] = score_item['score']
                    self.signals[sig_num]['ai_reasoning'] = score_item['reasoning']
            
            self.signals.sort(key=lambda x: x.get('ai_score', 0), reverse=True)
            
            for rank, signal in enumerate(self.signals, 1):
                signal['rank'] = rank
            
            self.top_signal_info = scores_data.get('top_signal', {})
            
            print(f"✅ AI scored {len(self.signals)} signals")
            print(f"🏆 Top Signal: #{self.top_signal_info.get('signal_number', 'N/A')}")
            
        except Exception as e:
            print(f"❌ AI Scoring Error: {str(e)}")
            for signal in self.signals:
                signal['ai_score'] = 50
                signal['ai_reasoning'] = 'AI scoring failed'
    
    def save_locally(self):
        """Save all analysis data to local folder"""
        print(f"\n💾 Saving files locally to: {self.date_folder}")
        
        try:
            current = self.data.iloc[-1]
            
            # Save technical data CSV
            csv_filename = self._generate_filename('technical_data', 'csv')
            csv_path = os.path.join(self.date_folder, csv_filename)
            self.data.to_csv(csv_path)
            print(f"✅ Saved: {csv_filename}")
            
            # Save signals JSON with AI scores
            signals_data = {
                'symbol': self.symbol,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'date': datetime.now().strftime('%Y-%m-%d'),
                'price': float(current['Close']),
                'signals': self.signals,
                'signal_count': len(self.signals),
                'top_signal_info': getattr(self, 'top_signal_info', {})
            }
            
            json_filename = self._generate_filename('signals', 'json')
            json_path = os.path.join(self.date_folder, json_filename)
            with open(json_path, 'w') as f:
                json.dump(signals_data, f, indent=2)
            print(f"✅ Saved: {json_filename}")
            
            # Save ranked signals report
            ranked_filename = self._generate_filename('ranked_signals', 'txt')
            ranked_path = os.path.join(self.date_folder, ranked_filename)
            with open(ranked_path, 'w') as f:
                f.write(self.generate_ranked_report())
            print(f"✅ Saved: {ranked_filename}")
            
            print(f"\n✅ All files saved to: {self.date_folder}")
            return True
            
        except Exception as e:
            print(f"❌ Local Save Error: {str(e)}")
            return False
    
    def generate_ranked_report(self):
        """Generate ranked signals report"""
        current = self.data.iloc[-1]
        
        report = f"""
{'='*80}
RANKED SIGNALS REPORT - {self.symbol}
{'='*80}

Price: ${current['Close']:.2f} | Change: {current['Price_Change']:.2f}%
Date: {current.name.strftime('%Y-%m-%d')}

"""
        if hasattr(self, 'top_signal_info') and self.top_signal_info:
            report += f"""🏆 TOP SIGNAL: #{self.top_signal_info.get('signal_number', 'N/A')}
{self.top_signal_info.get('why', 'N/A')}

"""
        
        report += f"{'='*80}\nALL SIGNALS (Ranked by AI):\n{'='*80}\n\n"
        
        for signal in self.signals:
            score = signal.get('ai_score', 'N/A')
            rank = signal.get('rank', '?')
            
            if isinstance(score, (int, float)):
                indicator = "🔥" if score >= 80 else "⚡" if score >= 60 else "📊" if score >= 40 else "⚠️"
            else:
                indicator = "❓"
            
            report += f"""#{rank} {indicator} SCORE: {score}/100
Signal: {signal['signal']}
Description: {signal['desc']}
AI Analysis: {signal.get('ai_reasoning', 'N/A')}
{'-'*80}

"""
        
        return report
    
    def display_results(self):
        """Display formatted results in notebook"""
        from IPython.display import display, HTML
        
        current = self.data.iloc[-1]
        
        # Header
        html = f"""
        <div style="border: 2px solid #4CAF50; padding: 20px; border-radius: 10px; margin: 20px 0;">
            <h2 style="color: #4CAF50;">📊 {self.symbol} Technical Analysis</h2>
            <p><strong>Price:</strong> ${current['Close']:.2f} | 
               <strong>Change:</strong> {current['Price_Change']:.2f}% | 
               <strong>Date:</strong> {current.name.strftime('%Y-%m-%d')}</p>
        </div>
        """
        
        # Key Indicators
        html += f"""
        <div style="background: #f5f5f5; padding: 15px; border-radius: 5px; margin: 20px 0;">
            <h3>📈 Key Indicators</h3>
            <table style="width: 100%;">
                <tr>
                    <td><strong>RSI:</strong> {current['RSI']:.1f}</td>
                    <td><strong>MACD:</strong> {current['MACD']:.4f}</td>
                    <td><strong>ADX:</strong> {current['ADX']:.1f}</td>
                </tr>
                <tr>
                    <td><strong>Stochastic:</strong> {current['Stoch_K']:.1f}</td>
                    <td><strong>MFI:</strong> {current['MFI']:.1f}</td>
                    <td><strong>Volatility:</strong> {current['Volatility']:.1f}%</td>
                </tr>
            </table>
        </div>
        """
        
        # Top Signal
        if hasattr(self, 'top_signal_info') and self.top_signal_info:
            html += f"""
            <div style="background: #fff3cd; padding: 15px; border-radius: 5px; margin: 20px 0; border-left: 5px solid #ffc107;">
                <h3>🏆 Top Signal by AI</h3>
                <p><strong>Signal #{self.top_signal_info.get('signal_number', 'N/A')}</strong></p>
                <p>{self.top_signal_info.get('why', 'N/A')}</p>
            </div>
            """
        
        # Top 5 Signals
        html += """
        <div style="margin: 20px 0;">
            <h3>🎯 Top 5 AI-Ranked Signals</h3>
            <table style="width: 100%; border-collapse: collapse;">
                <thead style="background: #4CAF50; color: white;">
                    <tr>
                        <th style="padding: 10px; text-align: left;">Rank</th>
                        <th style="padding: 10px; text-align: left;">Score</th>
                        <th style="padding: 10px; text-align: left;">Signal</th>
                        <th style="padding: 10px; text-align: left;">AI Reasoning</th>
                    </tr>
                </thead>
                <tbody>
        """
        
        for i, sig in enumerate(self.signals[:5], 1):
            score = sig.get('ai_score', 'N/A')
            indicator = "🔥" if isinstance(score, (int, float)) and score >= 80 else "⚡" if isinstance(score, (int, float)) and score >= 60 else "📊"
            
            html += f"""
                <tr style="border-bottom: 1px solid #ddd;">
                    <td style="padding: 10px;">#{i}</td>
                    <td style="padding: 10px;">{indicator} {score}/100</td>
                    <td style="padding: 10px;"><strong>{sig['signal']}</strong><br><small>{sig['desc']}</small></td>
                    <td style="padding: 10px;"><small>{sig.get('ai_reasoning', 'N/A')[:80]}...</small></td>
                </tr>
            """
        
        html += """
                </tbody>
            </table>
        </div>
        """
        
        # Summary Stats
        bullish = sum(1 for s in self.signals if 'BULLISH' in s['strength'])
        bearish = sum(1 for s in self.signals if 'BEARISH' in s['strength'])
        
        html += f"""
        <div style="background: #e3f2fd; padding: 15px; border-radius: 5px; margin: 20px 0;">
            <h3>📊 Signal Summary</h3>
            <p><strong>Total Signals:</strong> {len(self.signals)}</p>
            <p><strong>Bullish:</strong> {bullish} | <strong>Bearish:</strong> {bearish}</p>
            <p><strong>Overall Bias:</strong> {'🟢 BULLISH' if bullish > bearish else '🔴 BEARISH' if bearish > bullish else '🟡 NEUTRAL'}</p>
        </div>
        """
        
        display(HTML(html))
        
        return self

print("✅ Cell 1 Complete: TechnicalAnalyzer class loaded")


✅ Cell 1 Complete: TechnicalAnalyzer class loaded


In [6]:
# ===================================================================================
# CELL 2: CONFIGURATION & RUN ANALYSIS
# ===================================================================================
# Configure your analysis and execute the scanner

# ============== CONFIGURATION ==============
SYMBOL = 'Roku'           # Change to any ticker: AAPL, TSLA, NVDA, etc.
PERIOD = '1mo'             # Options: '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', 'max'
GEMINI_API_KEY = None     # Set to your API key or use: os.getenv('GEMINI_API_KEY')

# Optional: Uncomment to set API key directly
# GEMINI_API_KEY = 'your-api-key-here'

# Or use environment variable (recommended):
import os
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# ===========================================

print("=" * 80)
print(f"🚀 TECHNICAL SCANNER: {SYMBOL}")
print("=" * 80)

# Initialize analyzer
analyzer = TechnicalAnalyzer(
    symbol=SYMBOL,
    period=PERIOD,
    gemini_api_key=GEMINI_API_KEY
)

# Execute analysis pipeline
print("\n📊 Step 1: Fetching market data...")
analyzer.fetch_data()

print("\n🔧 Step 2: Calculating indicators...")
analyzer.calculate_indicators()

print("\n🎯 Step 3: Detecting signals...")
analyzer.detect_signals()

print("\n🤖 Step 4: AI ranking signals (1-100)...")
if GEMINI_API_KEY:
    analyzer.rank_signals_with_ai()
else:
    print("⚠️  No Gemini API key provided. Set GEMINI_API_KEY for AI ranking.")
    print("Get API key from: https://aistudio.google.com/app/apikey")

print("\n💾 Step 5: Saving results...")
analyzer.save_locally()

print("\n" + "=" * 80)
print("✅ ANALYSIS COMPLETE!")
print("=" * 80)
print(f"📂 Results saved to: {analyzer.date_folder}")

# Store analyzer for Cell 3
_analyzer = analyzer


🚀 TECHNICAL SCANNER: Roku
📁 Using existing folder: technical_analysis_data/2025-11-02

📊 Step 1: Fetching market data...
📊 Fetching data for Roku...
✅ Fetched 23 days of data

🔧 Step 2: Calculating indicators...

🔧 Calculating Technical Indicators...
✅ All indicators calculated

🎯 Step 3: Detecting signals...

🎯 Scanning for Technical Alerts...
✅ Detected 7 Active Signals

🤖 Step 4: AI ranking signals (1-100)...

🤖 AI is scoring all signals (1-100)...
✅ AI scored 7 signals
🏆 Top Signal: #6

💾 Step 5: Saving results...

💾 Saving files locally to: technical_analysis_data/2025-11-02
✅ Saved: 2025-11-02-Roku-technical_data-204115.csv
✅ Saved: 2025-11-02-Roku-signals-204115.json
✅ Saved: 2025-11-02-Roku-ranked_signals-204115.txt

✅ All files saved to: technical_analysis_data/2025-11-02

✅ ANALYSIS COMPLETE!
📂 Results saved to: technical_analysis_data/2025-11-02


In [7]:
# ===================================================================================
# CELL 3: VIEW RESULTS & EXPORT
# ===================================================================================
# Display results and access data for further analysis

# Display formatted results
print("=" * 80)
print("📊 DISPLAYING RESULTS")
print("=" * 80)

_analyzer.display_results()

# Access the data
print("\n" + "=" * 80)
print("📋 DATA ACCESS")
print("=" * 80)

# Get the full technical data DataFrame
technical_data = _analyzer.data
print(f"\n✅ Technical data available in 'technical_data' variable")
print(f"   Shape: {technical_data.shape}")
print(f"   Columns: {len(technical_data.columns)} indicators")

# Get signals list
signals_list = _analyzer.signals
print(f"\n✅ Signals available in 'signals_list' variable")
print(f"   Total signals: {len(signals_list)}")

# Show top 10 signals
print("\n🏆 TOP 10 AI-RANKED SIGNALS:")
print("=" * 80)
for i, sig in enumerate(signals_list[:10], 1):
    score = sig.get('ai_score', 'N/A')
    indicator = "🔥" if isinstance(score, (int, float)) and score >= 80 else "⚡" if isinstance(score, (int, float)) and score >= 60 else "📊"
    
    print(f"\n#{i} {indicator} [{score}/100] {sig['signal']}")
    print(f"   Description: {sig['desc']}")
    print(f"   Category: {sig['category']} | Strength: {sig['strength']}")
    print(f"   AI Reasoning: {sig.get('ai_reasoning', 'N/A')}")

# Export options
print("\n" + "=" * 80)
print("📤 EXPORT OPTIONS")
print("=" * 80)
print(f"\n1. Files saved locally in: {_analyzer.date_folder}")
print(f"   - Technical data CSV (all indicators)")
print(f"   - Signals JSON (with AI scores)")
print(f"   - Ranked signals TXT report")

print("\n2. Access data in notebook:")
print("   - technical_data: Full DataFrame with all indicators")
print("   - signals_list: List of all signals with AI scores")
print("   - _analyzer: Full analyzer object")

# Create signals DataFrame for easy viewing
signals_df = pd.DataFrame([
    {
        'Rank': sig.get('rank', '?'),
        'Score': sig.get('ai_score', 'N/A'),
        'Signal': sig['signal'],
        'Description': sig['desc'],
        'Category': sig['category'],
        'Strength': sig['strength'],
        'AI_Reasoning': sig.get('ai_reasoning', 'N/A')[:60] + '...'
    }
    for sig in signals_list
])

print("\n✅ Signals DataFrame created: 'signals_df'")
print("\nPreview:")
display(signals_df.head(10))

# Show latest price data
print("\n📈 LATEST PRICE DATA:")
print("=" * 80)
latest = technical_data.iloc[-1]
print(f"Date: {latest.name.strftime('%Y-%m-%d')}")
print(f"Close: ${latest['Close']:.2f}")
print(f"Change: {latest['Price_Change']:.2f}%")
print(f"Volume: {latest['Volume']:,.0f}")
print(f"RSI: {latest['RSI']:.1f}")
print(f"MACD: {latest['MACD']:.4f}")
print(f"ADX: {latest['ADX']:.1f}")

print("\n" + "=" * 80)
print("✅ CELL 3 COMPLETE - Results ready for analysis!")
print("=" * 80)

# Optional: Plot some indicators
print("\n📊 OPTIONAL: Uncomment code below to plot indicators")
print("""
# Uncomment to plot price + indicators:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Price + MAs
technical_data[['Close', 'SMA_10', 'SMA_20', 'SMA_50']].tail(100).plot(ax=axes[0])
axes[0].set_title(f'{SYMBOL} Price + Moving Averages')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# RSI
technical_data['RSI'].tail(100).plot(ax=axes[1], color='purple')
axes[1].axhline(70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(30, color='g', linestyle='--', alpha=0.5)
axes[1].set_title('RSI')
axes[1].set_ylabel('RSI')
axes[1].grid(True)

# MACD
technical_data[['MACD', 'MACD_Signal']].tail(100).plot(ax=axes[2])
axes[2].set_title('MACD')
axes[2].set_ylabel('MACD')
axes[2].legend()
axes[2].grid(True)

# Volume
technical_data['Volume'].tail(100).plot(ax=axes[3], kind='bar', color='steelblue')
axes[3].set_title('Volume')
axes[3].set_ylabel('Volume')
axes[3].grid(True)

plt.tight_layout()
plt.show()
""")

📊 DISPLAYING RESULTS



📋 DATA ACCESS

✅ Technical data available in 'technical_data' variable
   Shape: (23, 60)
   Columns: 60 indicators

✅ Signals available in 'signals_list' variable
   Total signals: 7

🏆 TOP 10 AI-RANKED SIGNALS:

#1 🔥 [95/100] VOLUME BREAKOUT
   Description: High volume + price up
   Category: VOLUME | Strength: STRONG BULLISH
   AI Reasoning: High volume combined with a price increase signals a strong breakout, indicating continued upward momentum. Highly actionable, reliable, excellent timing, and great risk/reward. It's a very compelling signal. The best signal overall.

#2 🔥 [90/100] EXTREME VOLUME 3X
   Description: Vol: 15,526,800
   Category: VOLUME | Strength: VERY SIGNIFICANT
   AI Reasoning: Extreme volume of 3x the average confirms significant buying pressure and validates the price movement. Highly actionable, reliable, excellent timing, and good risk/reward. This signal is very strong. This is a key signal alongside the bull cross.

#3 🔥 [88/100] LARGE GAIN
   Descriptio

,Rank,Score,Signal,Description,Category,Strength,AI_Reasoning
0,1,95,VOLUME BREAKOUT,High volume + price up,VOLUME,STRONG BULLISH,High volume combined with a price increase sig...
1,2,90,EXTREME VOLUME 3X,"Vol: 15,526,800",VOLUME,VERY SIGNIFICANT,Extreme volume of 3x the average confirms sign...
2,3,88,LARGE GAIN,+6.1% today,PRICE_ACTION,STRONG BULLISH,A large gain of +6.1% indicates strong buying ...
3,4,85,10/20 EMA BULL CROSS,10 EMA crossed above 20 EMA,MA_CROSS,BULLISH,10/20 EMA bull cross is a classic bullish sign...
4,5,78,VOLUME SPIKE 2X,"Vol: 15,526,800",VOLUME,SIGNIFICANT,Volume spike of 2x the average is a strong ind...
5,6,72,RSI OVERBOUGHT,RSI at 71.4,RSI,BEARISH,"RSI overbought suggests a potential pullback, ..."
6,7,65,AT UPPER BB,Price at $105.00,BOLLINGER,BEARISH,Price at the upper Bollinger Band suggests a p...



📈 LATEST PRICE DATA:
Date: 2025-10-31
Close: $106.13
Change: 6.10%
Volume: 15,526,800
RSI: 71.4
MACD: -0.1260
ADX: nan

✅ CELL 3 COMPLETE - Results ready for analysis!

📊 OPTIONAL: Uncomment code below to plot indicators

# Uncomment to plot price + indicators:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(4, 1, figsize=(15, 12))

# Price + MAs
technical_data[['Close', 'SMA_10', 'SMA_20', 'SMA_50']].tail(100).plot(ax=axes[0])
axes[0].set_title(f'{SYMBOL} Price + Moving Averages')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True)

# RSI
technical_data['RSI'].tail(100).plot(ax=axes[1], color='purple')
axes[1].axhline(70, color='r', linestyle='--', alpha=0.5)
axes[1].axhline(30, color='g', linestyle='--', alpha=0.5)
axes[1].set_title('RSI')
axes[1].set_ylabel('RSI')
axes[1].grid(True)

# MACD
technical_data[['MACD', 'MACD_Signal']].tail(100).plot(ax=axes[2])
axes[2].set_title('MACD')
axes[2].set_ylabel('MACD')
axes[2].legend()
axes[2].grid(True)

# Volume
te